# US Treasury Yield Analysis
This notebook compares different optimization engines (MOSEK, Gurobi, CPLEX) for minimizing funding costs in US Treasury bonds using real yield data from 2024.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO

# Download and load the data
url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_real_yield_curve&field_tdr_date_value=2024"
response = requests.get(url)
data = response.text

# Parse the data into a DataFrame
# Assuming the data is structured in a CSV-like format (example, you may need to adjust for actual format)
df = pd.read_csv(StringIO(data))

# View data structure
df.head()

In [2]:
# Convert date to datetime format and set it as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Calculate the relevant data columns (e.g., funding rates, haircuts, transfer fees)
# Assuming 'Rate' column corresponds to treasury yields and we derive funding costs from them
df['FundingCost'] = df['Rate'] * (1 + 0.01)  # Add a simple transfer fee for example

# Visualize funding costs
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['FundingCost'], label='Funding Costs')
plt.xlabel('Date')
plt.ylabel('Funding Cost')
plt.title('Funding Costs for US Treasury 2024')
plt.legend()
plt.show()

In [3]:
from mosek.fusion import *

# Define the optimization problem variables
def optimization_model(data):
    with Model('treasury_optimization') as M:
        # Variables: investment amounts in different treasury bonds
        amounts = M.variable('amounts', 12, Domain.greaterThan(0.0))  # Assuming 12 bonds in 12 months
        
        # Objective function: Minimize the funding cost
        cost = Expr.dot(data['FundingCost'].values, amounts)
        M.objective('minimize_cost', ObjectiveSense.Minimize, cost)
        
        # Constraints (example: haircuts, limits on borrowing)
        M.constraint('max_haircut', Expr.sum(amounts), Domain.lessThan(100))
        
        # Solve the model
        M.solve()
        return M

# Solve using MOSEK
mo_model = optimization_model(df)
print("Optimal investment amounts:", mo_model.variable('amounts').level())

In [4]:
# Gurobi example
from gurobipy import *

def gurobi_model(data):
    model = Model("treasury_optimization")
    
    # Decision variables
    amounts = model.addVars(12, lb=0, name="amounts")
    
    # Objective: Minimize funding cost
    model.setObjective(quicksum(data['FundingCost'][i] * amounts[i] for i in range(12)), GRB.MINIMIZE)
    
    # Constraints (similar to the MOSEK case)
    model.addConstr(quicksum(amounts[i] for i in range(12)) <= 100, "max_haircut")
    
    # Optimize
    model.optimize()
    return model.objVal, [amounts[i].x for i in range(12)]

# Solve using Gurobi
gurobi_cost, gurobi_amounts = gurobi_model(df)
print("Gurobi optimal investment:", gurobi_amounts)

In [5]:
# Plot comparison
solvers = ['MOSEK', 'Gurobi']
costs = [mo_model.objective().value(), gurobi_cost]  # Add CPLEX cost here

plt.figure(figsize=(10, 6))
plt.bar(solvers, costs)
plt.title('Comparison of Optimization Engines')
plt.ylabel('Minimized Funding Cost')
plt.show()